In [12]:
from use_translator import *

In [13]:
def create_query(disease_id):
    query_graph = {'nodes': {'disease': {'ids': [disease_id]}},
                   'edges': {}}
    pnode = 'disease'
    cnode = 'chemical'
    query_graph['nodes'][cnode] = {'categories': ['biolink:ChemicalEntity']}
    query_graph['edges']['t_edge'] = {'object': pnode, 'subject': cnode, 'predicate':'biolink:treats', 'knowledge_type': 'inferred'}
    m = {'message':{'query_graph':query_graph}}
    return m

In [14]:
tq = create_query("MONDO:0004975")

In [15]:
result = post('local_robokop','http://localhost:4868/robokop/query',tq)

local_robokop returned in 0:01:03.389155s
Num Results: 613


In [7]:
with open('infer_robokop.json','w') as outf:
    json.dump(result,outf)

In [135]:
x = post('nn','https://nodenormalization-sri.renci.org/1.2/response',result)

nn returned in 0:00:08.395327s
Num Results: 0


In [142]:
from copy import deepcopy
fixedresults = deepcopy(result)
fixedresults['message']['knowledge_graph']['nodes'] = x['message']['knowledge_graph']['nodes']

In [143]:
with open('robokop_AD.json','w') as outf:
    json.dump(fixedresults,outf,indent=4)

In [138]:
printjson(result['message']['results'][1])

{
    "node_bindings": {
        "disease": [
            {
                "id": "MONDO:0004975"
            }
        ],
        "chemical": [
            {
                "id": "PUBCHEM.COMPOUND:2355"
            }
        ]
    },
    "edge_bindings": {
        "_dummy_edge0": [
            {
                "id": "edge_1714",
                "attributes": [
                    {
                        "attribute_type_id": "biolink:has_numeric_value",
                        "value": 1,
                        "value_type_id": "EDAM:data_1669",
                        "original_attribute_name": "weight",
                        "value_url": null,
                        "attribute_source": null,
                        "description": null,
                        "attributes": null
                    }
                ]
            }
        ],
        "_dummy_edge1": [
            {
                "id": "edge_1715",
                "attributes": [
                    {
       

In [139]:
printjson(result['message']['knowledge_graph']['edges']['edge_1716'])

{
    "subject": "PUBCHEM.COMPOUND:2355",
    "object": "MONDO:0004975",
    "predicate": "biolink:ameliorates",
    "attributes": [
        {
            "original_attribute_name": "weight",
            "attribute_type_id": "biolink:has_numeric_value",
            "value": 1,
            "value_type_id": "EDAM:data_1669",
            "value_url": null,
            "attribute_source": null
        }
    ]
}


In [140]:
printjson(x['message']['knowledge_graph']['edges']['edge_1716'])

KeyError: 'edge_1716'

In [63]:
printjson(result['message']['results'][1])

{
    "node_bindings": {
        "disease": [
            {
                "id": "MONDO:0004975"
            }
        ],
        "chemical": [
            {
                "id": "PUBCHEM.COMPOUND:5565"
            }
        ],
        "_dummy_node_0": [
            {
                "id": "NCBIGene:351"
            }
        ]
    },
    "edge_bindings": {
        "_dummy_edge0": [
            {
                "id": "edge_4"
            }
        ],
        "_dummy_edge1": [
            {
                "id": "edge_5"
            }
        ],
        "_dummy_edge2": [
            {
                "id": "edge_6"
            }
        ],
        "_dummy_edge3": [
            {
                "id": "edge_7"
            }
        ]
    }
}


In [93]:
with open('infer_AD.json','w') as outf:
    json.dump(result,outf,indent=4)

In [94]:
printjson(result['message']['results'][0])

{
    "node_bindings": {
        "disease": [
            {
                "id": "MONDO:0004975",
                "qnode_id": "MONDO:0004975"
            }
        ],
        "chemical": [
            {
                "id": "PUBCHEM.COMPOUND:187"
            }
        ]
    },
    "edge_bindings": {
        "_dummy_edge_0": [
            {
                "id": "ee36185cb013",
                "attributes": [
                    {
                        "attribute_type_id": "biolink:has_numeric_value",
                        "value": 0.4697013917693704,
                        "value_type_id": "EDAM:data_1669",
                        "original_attribute_name": "weight",
                        "value_url": null,
                        "attribute_source": null,
                        "description": null,
                        "attributes": null
                    }
                ]
            },
            {
                "id": "0b9c5caed1cd",
                "attributes":

In [97]:
for r in result['message']['results']:
    nn = len(r['node_bindings'])
    if nn > 2:
        print(nn)

In [110]:
#changed backend to use 25 rules
result_25 = post('local_aragorn','http://localhost:4868/aragorn/query',tq)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [105]:
for i,r in enumerate(result_25['message']['results']):
    nn = len(r['node_bindings'])
    if nn > 2:
        print(i,nn)

In [ ]:
with open('infer_AD_25.json','w') as outf:
    json.dump(result_25,outf,indent=4)

In [130]:
from string import Template
rules=[
    Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:has_phenotype"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:increases_molecular_modification_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:affects_degradation_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:affects_degradation_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:affects_stability_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:biomarker_for"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:similar_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:has_phenotype"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:increases_molecular_interaction"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:subclass_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:subclass_of"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "f", "object": "$disease", "predicates": ["biolink:subclass_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:decreases_response_to"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:has_phenotype"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:has_substrate"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "e", "object": "$chemical", "predicates": ["biolink:subclass_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:genetic_association"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:increases_uptake_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_stability_of"]}, "edge_1": {"subject": "f", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_stability_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:affects_abundance_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:has_phenotype"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:increases_splicing_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_molecular_modification_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:similar_to"]}, "edge_1": {"subject": "f", "object": "$disease", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "f", "predicates": ["biolink:similar_to"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "f", "object": "$disease", "predicates": ["biolink:associated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:associated_with"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:similar_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:expression_decreased_by"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:has_phenotype"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "e", "object": "$chemical", "predicates": ["biolink:similar_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:affects_localization_of"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:causes"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:increases_response_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$disease", "predicates": ["biolink:has_phenotype"]}, "edge_1": {"subject": "e", "object": "$chemical", "predicates": ["biolink:increases_response_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}, "edge_1": {"subject": "f", "object": "$chemical", "predicates": ["biolink:increases_localization_of"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_molecular_modification_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_metabolic_processing_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_response_to"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_localization_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$chemical", "predicates": ["biolink:affects_localization_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:causes"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "e": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "e", "object": "$chemical", "predicates": ["biolink:expression_increased_by"]}, "edge_1": {"subject": "e", "object": "$disease", "predicates": ["biolink:has_phenotype"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "f", "predicates": ["biolink:decreases_localization_of"]}, "edge_1": {"subject": "$disease", "object": "f", "predicates": ["biolink:genetic_association"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}, "f": {"categories": ["biolink:NamedThing"]}}, "edges": {"edge_0": {"subject": "f", "object": "$disease", "predicates": ["biolink:has_part"]}, "edge_1": {"subject": "$chemical", "object": "f", "predicates": ["biolink:treats"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:affects"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "$chemical", "predicates": ["biolink:affects"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:biomarker_for"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:negatively_correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:positively_correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:biomarker_for"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes_adverse_event"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:biomarker_for"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:positively_correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:ameliorates"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:positively_correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:negatively_correlated_with"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "$chemical", "predicates": ["biolink:affects"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes_adverse_event"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$disease", "object": "$chemical", "predicates": ["biolink:affects"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:affects"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:negatively_correlated_with"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes_adverse_event"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:related_to"]}}}}'),
Template('{"query_graph": {"nodes": {"$chemical": {"categories": ["biolink:ChemicalEntity"]}, "$disease": {"ids": ["$disease_id"]}}, "edges": {"edge_0": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes"]}, "edge_1": {"subject": "$chemical", "object": "$disease", "predicates": ["biolink:causes_adverse_event"]}}}}'),hl
]

In [132]:
disease = "MONDO:0004975"
dnode = 'disease'
cnode = 'chemical'
queries = []
nres = []
for t in rules:
    query = t.substitute(disease=dnode, chemical=cnode, disease_id=disease)
    qg = json.loads(query)
    m = {'message': qg}
    sres = post('strider-dev','https://strider-dev.apps.renci.org/1.2/query',m)
    queries.append(qg)
    nres.append(len(sres['message']['results']))

strider-dev returned in 0:00:35.200685s
Num Results: 0
strider-dev returned in 0:02:32.575194s
Num Results: 120
strider-dev returned in 0:01:12.532763s
Num Results: 100
strider-dev returned in 0:01:46.039396s
Num Results: 1
strider-dev returned in 0:00:24.303511s
Num Results: 1067
strider-dev returned in 0:00:19.224724s
Num Results: 0
strider-dev returned in 0:12:28.305670s
Num Results: 32342
strider-dev returned in 0:03:51.188393s
Num Results: 19266
strider-dev returned in 0:03:49.405493s
Num Results: 20834
strider-dev returned in 0:00:36.347965s
Num Results: 0
strider-dev returned in 0:01:49.955766s
Num Results: 6
strider-dev returned in 0:13:03.422022s
Num Results: 33431
strider-dev returned in 0:01:13.026265s
Num Results: 34


KeyboardInterrupt: 

In [125]:
edges = sres['message']['knowledge_graph']['edges']
preds = set([edges[e]['predicate'] for e in edges])
for p in preds:
    print(p)

biolink:overlaps
biolink:acts_upstream_of
biolink:capable_of
biolink:actively_involved_in
biolink:participates_in
biolink:acts_upstream_of_or_within
biolink:active_in
biolink:has_real_world_evidence_of_association_with
biolink:condition_associated_with_gene
biolink:has_phenotype
biolink:related_to
biolink:acts_upstream_of_or_within_positive_effect
biolink:acts_upstream_of_or_within_negative_effect
biolink:genetic_association
biolink:acts_upstream_of_negative_effect
biolink:gene_associated_with_condition
biolink:acts_upstream_of_positive_effect
biolink:increases_molecular_modification_of
biolink:enables


In [126]:
printjson(queries[0])

{
    "query_graph": {
        "nodes": {
            "chemical": {
                "categories": [
                    "biolink:ChemicalEntity"
                ]
            },
            "disease": {
                "ids": [
                    "MONDO:0004975"
                ]
            },
            "f": {
                "categories": [
                    "biolink:NamedThing"
                ]
            }
        },
        "edges": {
            "edge_0": {
                "subject": "f",
                "object": "disease",
                "predicate": "biolink:has_phenotype"
            },
            "edge_1": {
                "subject": "chemical",
                "object": "f",
                "predicate": "biolink:increases_molecular_modification_of"
            }
        }
    }
}


In [128]:
reg_strider = strider({'message':queries[0]})

strider returned in 0:02:49.288431s
Num Results: 1403


In [129]:
edges = reg_strider['message']['knowledge_graph']['edges']
preds = set([edges[e]['predicate'] for e in edges])
for p in preds:
    print(p)

biolink:overlaps
biolink:acts_upstream_of_or_within_negative_effect
biolink:acts_upstream_of
biolink:genetic_association
biolink:acts_upstream_of_positive_effect
biolink:acts_upstream_of_negative_effect
biolink:capable_of
biolink:active_in
biolink:enables
biolink:participates_in
biolink:related_to
biolink:acts_upstream_of_or_within
biolink:actively_involved_in
biolink:acts_upstream_of_or_within_positive_effect


In [118]:
result_1_rule = post('local_aragorn','http://localhost:4868/aragorn/query',tq)

local_aragorn returned in 0:08:09.717948s
Num Results: 1306


In [119]:
printjson(result_1_rule['message']['results'][0])

{
    "node_bindings": {
        "disease": [
            {
                "id": "MONDO:0004975"
            }
        ],
        "chemical": [
            {
                "id": "NCBIGene:7249"
            }
        ],
        "_dummy_node_0": [
            {
                "id": "UBERON:0000955"
            }
        ]
    },
    "edge_bindings": {
        "_dummy_edge_0": [
            {
                "id": "5deb145c4b63",
                "attributes": [
                    {
                        "attribute_type_id": "biolink:has_numeric_value",
                        "value": 0.2679491924311228,
                        "value_type_id": "EDAM:data_1669",
                        "original_attribute_name": "weight",
                        "value_url": null,
                        "attribute_source": null,
                        "description": null,
                        "attributes": null
                    }
                ]
            }
        ],
        "_dummy_ed

In [121]:
with open('infer_AD_1.json','w') as outf:
    json.dump(result_1_rule,outf,indent=4)